In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [20]:
import pandas as pd
df = pd.read_csv('/content/drive/MyDrive/MyDrive/new_pred0.csv', encoding='utf-8')

In [21]:
import pandas as pd
import numpy as np
from keras.models import Sequential
from keras.layers import Dense, SimpleRNN, GRU
from tensorflow.keras.losses import Huber
# from keras.optimizers import SGD
from keras.layers import Dropout
from keras.optimizers import Adam
from sklearn.preprocessing import MinMaxScaler




In [22]:
wpvna=df['prodname'].unique()
wpvna=[wpvna]
wpvna

[array(['AE', 'CSA4000', 'CSA4000(PCA)', 'CSA5000', 'PEMA-500FR',
        'PEMA-580FX', 'PEMA-CR1000', 'PEMA-CSA5000', 'PEMA-HR1000',
        'PEMA-HR1000S', 'PEMA-HR1500', 'PEMA-PCM2000', 'PEMA-PCM2000B',
        'PEMA-PCR3000E', 'PEMA-PCR3000N', 'PEMA-PR1000', 'PEMA-PR2000',
        'PEMA-SN400', 'PEMA-SP1000', 'PEMA-SPR', 'PEMA-SR2000',
        'PEMA-SR2000A', 'PEMA-SR3000F', 'PEMA-SR5000F', 'PR1000',
        'SRE-110', 'SRE-200'], dtype=object)]

In [30]:
product_df = {}
product_df2 = {}
for product_name in wpvna[0]:
    # 특정 제품에 대한 데이터를 필터링합니다.
    filtered_data = df[df['prodname'] == product_name].copy()  # .copy() 추가
    filtered_data.drop(['rn', 'prodname', 'order_quant'], axis=1, inplace=True)
    # 인덱스를 재설정하고 'week'를 인덱스로 설정합니다.
    filtered_data.reset_index(drop=True, inplace=True)
    filtered_data.set_index('week', inplace=True)
    filtered_data.rename(columns={'tem_avg': '온도', 'hum_avg': '습도', 'sold_quant': '판매수량'}, inplace=True)
    product_df2[product_name] = filtered_data.copy()
    scaler = MinMaxScaler()
    filtered_data[['판매수량', '온도', '습도', '국내건설수주액', '국내기성액']] = scaler.fit_transform(filtered_data[['판매수량', '온도', '습도', '국내건설수주액', '국내기성액']])
    # 이제 filtered_data는 특정 제품에 대한 스케일링된 데이터를 포함합니다.
    # 이제 해당 제품명을 key로 하고 데이터프레임을 value로 하는 사전에 저장합니다.
    product_df[product_name] = filtered_data
    product_df[product_name].index = pd.to_datetime(product_df[product_name].index)


In [31]:
  product_df['AE']

,온도,습도,판매수량,국내건설수주액,국내기성액
week,,,,,
2018-11-05,0.689975,1.000000,0.323341,0.193740,0.368437
2018-11-12,0.577928,0.627978,0.253284,0.193740,0.368437
2018-11-19,0.454928,0.767868,0.988885,0.193740,0.368437
2018-11-26,0.471778,0.778864,0.638599,0.193740,0.368437
2018-12-03,0.283067,0.485644,0.292119,0.651448,0.797761
...,...,...,...,...,...
2021-03-22,0.860994,0.695174,0.338161,0.414145,0.497289
2021-03-29,0.861837,0.693341,0.102391,0.414145,0.497289
2021-04-05,0.862679,0.690287,0.304480,0.466471,0.385309


In [32]:
product_df2['AE']

,온도,습도,판매수량,국내건설수주액,국내기성액
week,,,,,
2018-11-05,9.81,71.79,14560,11924951,11419899
2018-11-12,8.48,65.70,12480,11924951,11419899
2018-11-19,7.02,67.99,34320,11924951,11419899
2018-11-26,7.22,68.17,23920,11924951,11419899
2018-12-03,4.98,63.37,13633,21742503,13836199
...,...,...,...,...,...
2021-03-22,11.84,66.80,15000,16652501,12145097
2021-03-29,11.85,66.77,8000,16652501,12145097
2021-04-05,11.86,66.72,14000,17774868,11514860


In [14]:
scdf

,week,prodname,tem_avg,hum_avg,order_quant,sold_quant,국내건설수주액,국내기성액,rn
0,2018-11-05,AE,9.81,71.79,13830,14560,11924951,11419899,1
1,2018-11-05,CSA4000,9.81,71.79,15764,12480,11924951,11419899,1
2,2018-11-05,CSA4000(PCA),9.81,71.79,0,0,11924951,11419899,1
3,2018-11-05,CSA5000,9.81,71.79,112083,118560,11924951,11419899,1
4,2018-11-05,PEMA-500FR,9.81,71.79,36817,41600,11924951,11419899,1
...,...,...,...,...,...,...,...,...,...
3478,2021-04-19,PEMA-SR3000F,11.89,66.54,67075,70000,17774868,11514860,129
3479,2021-04-19,PEMA-SR5000F,11.89,66.54,0,0,17774868,11514860,129
3480,2021-04-19,PR1000,11.89,66.54,0,0,17774868,11514860,129
3481,2021-04-19,SRE-110,11.89,66.54,0,0,17774868,11514860,129


In [163]:
product_df.get('AE')

,판매수량,온도,습도,국내건설수주액,국내기성액
week,,,,,
2018-11-05,0.323341,0.689975,1.000000,0.193740,0.368437
2018-11-12,0.253284,0.577928,0.627978,0.193740,0.368437
2018-11-19,0.988885,0.454928,0.767868,0.193740,0.368437
2018-11-26,0.638599,0.471778,0.778864,0.193740,0.368437
2018-12-03,0.292119,0.283067,0.485644,0.651448,0.797761
...,...,...,...,...,...
2021-03-22,0.338161,0.860994,0.695174,0.414145,0.497289
2021-03-29,0.102391,0.861837,0.693341,0.414145,0.497289
2021-04-05,0.304480,0.862679,0.690287,0.466471,0.385309


In [102]:
from keras.optimizers import Adam
import pandas as pd
import numpy as np
from keras.models import Sequential
from keras.layers import Dense, SimpleRNN, GRU
from tensorflow.keras.losses import Huber
# from keras.optimizers import SGD
from keras.layers import Dropout


all_trainX = {}
all_trainY = {}
all_testX = {}
all_testY = {}


for product_name in wpvna[0]:
    filtered_data = product_df.get(product_name)
    filtered_data_array = filtered_data.values
    traind_dates = filtered_data.index

    cols = list(filtered_data.columns)


    n_future = 2

    n_past = 20
    trainX  = []
    trainY  = []
    for i in range(n_past , len(filtered_data_array)- n_future+1): # 2 ~ (len(데이터) - 2-1)
        trainX.append(filtered_data_array[i - n_past:i , 0:filtered_data_array.shape[1]])# [-2:3] , 0:len(데이터)
        trainY.append(filtered_data_array[i + n_future -1 : i + n_future , 0])
        all_trainX[product_name]=trainX
        all_trainY[product_name]=trainY

    trainX, trainY = np.array(trainX), np.array(trainY)



    testX = []
    testY = []

# 테스트 데이터의 시작 인덱스 계산
    test_start = len(filtered_data_array) - n_past - n_future + 1

    for i in range(test_start, len(filtered_data_array) - n_future + 1):
        testX.append(filtered_data_array[i - n_past:i, 0:filtered_data_array.shape[1]])
        testY.append(filtered_data_array[i + n_future - 1:i + n_future, 0])
        all_testX[product_name]=testX
        all_testY[product_name]=testY

    testX, testY = np.array(testX), np.array(testY)





In [108]:
all_trainX = {}
all_trainY = {}
all_testX = {}
all_testY = {}
all_history = {}
all_predcition = {}
all_mse = {}
all_r2 = {}

In [ ]:
all_trainX['AE']

In [194]:

for product_name in wpvna[0]:

    BATCH_SIZE = 32
    time_steps = n_past
    unit = 128

    my_GRU_model = Sequential() # 순차적으로 레이블 생성

    my_GRU_model.add(GRU(units=unit, return_sequences=True, activation='tanh')) # 유닛수설정,출력시퀀스반환, 탄젠트 함수이용

    my_GRU_model.add(Dropout(0.1)) # 드롭아웃 과적합 줄이기

    my_GRU_model.add(GRU(units=unit, return_sequences=True, activation='tanh'))

    my_GRU_model.add(Dropout(0.1))

    my_GRU_model.add(GRU(units=unit, return_sequences=True, activation='tanh'))

    my_GRU_model.add(Dropout(0.1))


    my_GRU_model.add(GRU(units=unit, activation='tanh'))


    my_GRU_model.add(Dense(units=1))


    my_optimizer = Adam(learning_rate=0.002)


    my_GRU_model.compile(optimizer=my_optimizer, loss='huber_loss', metrics=['mse'])




    history = my_GRU_model.fit(np.array(all_trainX[product_name]), np.array(all_trainY[product_name]).reshape(-1,1), epochs=100, validation_split=0.2, batch_size=BATCH_SIZE, verbose=0)

    my_GRU_model.save(f'/content/drive/MyDrive/MyDrive/GRU_MODEL/{product_name}_model.h5')

    # 저장된 모델을 딕셔너리에 추가
    all_history[product_name] =f'{product_name}_model.h5'


    predictions = my_GRU_model.predict(np.array(all_trainX[product_name]))  # 모델을 사용하여 예측
    all_predcition[product_name]=predictions

    mse = mean_squared_error(np.array(all_trainY[product_name]).reshape(-1,1), predictions)
    all_mse[product_name] = mse
    print(mse)

    r2 = r2_score(np.array(all_trainY[product_name]).reshape(-1,1), predictions)
    all_r2[product_name] = r2
    print(r2)



AttributeError: ignored

In [195]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import GRU, Dropout, Dense
from tensorflow.keras.optimizers import Adam
import numpy as np

BATCH_SIZE = 32
time_steps = n_past
unit = 128

all_history = {}  # Initialize an empty dictionary to store models' filenames

for product_name in wpvna[0]:
    my_GRU_model = Sequential()
    my_GRU_model.add(GRU(units=unit, return_sequences=True, activation='tanh'))
    my_GRU_model.add(Dropout(0.1))
    my_GRU_model.add(GRU(units=unit, return_sequences=True, activation='tanh'))
    my_GRU_model.add(Dropout(0.1))
    my_GRU_model.add(GRU(units=unit, return_sequences=True, activation='tanh'))
    my_GRU_model.add(Dropout(0.1))
    my_GRU_model.add(GRU(units=unit, activation='tanh'))
    my_GRU_model.add(Dense(units=1))
    my_optimizer = Adam(learning_rate=0.002)
    my_GRU_model.compile(optimizer=my_optimizer, loss='huber_loss', metrics=['mse'])

    history = my_GRU_model.fit(np.array(all_trainX[product_name]), np.array(all_trainY[product_name]).reshape(-1,1), epochs=100, validation_split=0.2, batch_size=BATCH_SIZE, verbose=0)

    model_filename = f'/content/drive/MyDrive/MyDrive/GRU_MODEL/{product_name}_model.h5'
    my_GRU_model.save(model_filename)  # Save the model

    all_history[product_name] = model_filename  # Store the model's filename in the dictionary


KeyboardInterrupt: ignored

In [153]:
from keras.models import load_model
my_loaded_GRU_model = load_model(f'/content/drive/MyDrive/MyDrive/gru_모델/PEMA-HR1000_model.h5')

for product_name in wpvna[0]:
    n_future = 14
    forcast_period_dates = pd.date_range('2021-04-19', periods = n_future, freq = '7d').tolist()

    forecast = my_GRU_model.predict(np.array(all_trainX[])[-n_future:])
    forecast_copies = np.repeat(forecast, filtered_data.shape[1],axis=-1) # np.repeat(a, n) a를 n개 복사, axis=-1 현재 배열의 마지막 axis
    y_pred_future = scaler.inverse_transform(forecast_copies)[:,0]

    forecast_dataes = []

    for time_i in forcast_period_dates:
         forecast_dataes.append(time_i.date())
    len_forecast = len(forecast[:,-1])#n_future 만큼의 길이
    forecast_dates = pd.date_range(forcast_period_dates[1], periods=len_forecast, freq = str('7d')).tolist()

    df_forecast = pd.DataFrame({'week': np.array(pd.date_range(forcast_period_dates[1],  periods=len_forecast, freq = str('7d')).tolist()), 'sold_quant':y_pred_future})
    df_forecast['week'] = pd.to_datetime(df_forecast['week'])

1/1 [==============================] - 0s 21ms/step


In [208]:
forecast

array([[0.48382124],
       [0.21360251],
       [0.13162816],
       [0.21762219],
       [0.43250632],
       [0.23455   ],
       [0.13935325],
       [0.41009444],
       [0.47522765],
       [0.2326619 ],
       [0.07998803],
       [0.24092907],
       [0.796682  ],
       [0.29501897]], dtype=float32)

In [209]:
forecast_copies

array([[0.48382124, 0.48382124, 0.48382124, 0.48382124, 0.48382124],
       [0.21360251, 0.21360251, 0.21360251, 0.21360251, 0.21360251],
       [0.13162816, 0.13162816, 0.13162816, 0.13162816, 0.13162816],
       [0.21762219, 0.21762219, 0.21762219, 0.21762219, 0.21762219],
       [0.43250632, 0.43250632, 0.43250632, 0.43250632, 0.43250632],
       [0.23455   , 0.23455   , 0.23455   , 0.23455   , 0.23455   ],
       [0.13935325, 0.13935325, 0.13935325, 0.13935325, 0.13935325],
       [0.41009444, 0.41009444, 0.41009444, 0.41009444, 0.41009444],
       [0.47522765, 0.47522765, 0.47522765, 0.47522765, 0.47522765],
       [0.2326619 , 0.2326619 , 0.2326619 , 0.2326619 , 0.2326619 ],
       [0.07998803, 0.07998803, 0.07998803, 0.07998803, 0.07998803],
       [0.24092907, 0.24092907, 0.24092907, 0.24092907, 0.24092907],
       [0.796682  , 0.796682  , 0.796682  , 0.796682  , 0.796682  ],
       [0.29501897, 0.29501897, 0.29501897, 0.29501897, 0.29501897]],
      dtype=float32)

In [210]:
y_pred_future

array([ 7620.1846,  3364.2395,  2073.1436,  3427.5496,  6811.9746,
        3694.1626,  2194.8137,  6458.9873,  7484.8354,  3664.425 ,
        1259.8114,  3794.6328, 12547.741 ,  4646.549 ], dtype=float32)

In [204]:
product_dataframes = {}

from keras.models import load_model
my_loaded_GRU_model = load_model(f'/content/drive/MyDrive/MyDrive/gru_모델/{productname}.h5')

n_future = 14
forcast_period_dates = pd.date_range('2021-04-19', periods = n_future, freq = '7d').tolist()

forecast = my_loaded_GRU_model.predict(np.array(all_trainX['AE'][-n_future:]))

forecast_copies = np.repeat(forecast, product_df.get('AE').shape[1],axis=-1) # np.repeat(a, n) a를 n개 복사, axis=-1 현재 배열의 마지막 axis

min_val = product_df2[product_name]['sold_quant'].min()
max_val = product_df2[product_name]['sold_quant'].max()

product_df2[product_name]['sold_quant'] = (product_df2[product_name]['sold_quant'] * (max_val - min_val)) + min_val
product_dataframes[product] = product_df2[product_name]

y_pred_future = scaler.inverse_transform(forecast_copies)[:,0]

forecast_dataes = []

for time_i in forcast_period_dates:
    forecast_dataes.append(time_i.date())
    len_forecast = len(forecast[:,-1])#n_future 만큼의 길이
    forecast_dates = pd.date_range(forcast_period_dates[1], periods=len_forecast, freq = str('7d')).tolist()

    df_forecast = pd.DataFrame({'week': np.array(pd.date_range(forcast_period_dates[1],  periods=len_forecast, freq = str('7d')).tolist()), 'sold_quant':y_pred_future})
    df_forecast['week'] = pd.to_datetime(df_forecast['week'])

1/1 [==============================] - 1s 1s/step


,week,sold_quant
0,2021-04-26,7620.184570
1,2021-05-03,3364.239502
2,2021-05-10,2073.143555
3,2021-05-17,3427.549561
4,2021-05-24,6811.974609
5,2021-05-31,3694.162598
6,2021-06-07,2194.813721
7,2021-06-14,6458.987305
8,2021-06-21,7484.835449
9,2021-06-28,3664.425049


In [167]:
product_df.get('PEMA-HR1000')

,판매수량,온도,습도,국내건설수주액,국내기성액
week,,,,,
2018-11-05,0.459210,0.689975,1.000000,0.193740,0.368437
2018-11-12,0.330117,0.577928,0.627978,0.193740,0.368437
2018-11-19,0.868747,0.454928,0.767868,0.193740,0.368437
2018-11-26,0.617238,0.471778,0.778864,0.193740,0.368437
2018-12-03,0.839288,0.283067,0.485644,0.651448,0.797761
...,...,...,...,...,...
2021-03-22,0.927837,0.860994,0.695174,0.414145,0.497289
2021-03-29,0.819652,0.861837,0.693341,0.414145,0.497289
2021-04-05,0.619549,0.862679,0.690287,0.466471,0.385309


In [174]:
forecast

array([[-0.05787401],
       [-0.05316967],
       [-0.04670049],
       [-0.04034681],
       [-0.03502016],
       [-0.03084862],
       [-0.02761944],
       [-0.02511154],
       [-0.02462105],
       [-0.0265068 ],
       [-0.03000837],
       [-0.03440547],
       [-0.03896206],
       [-0.04267142]], dtype=float32)

In [182]:
pp= np.array(all_trainX['PEMA-HR1000'])

In [185]:
pp

array([[[0.45920999, 0.68997473, 1.        , 0.19374021, 0.36843663],
        [0.3301167 , 0.57792755, 0.62797801, 0.19374021, 0.36843663],
        [0.86874731, 0.45492839, 0.76786805, 0.19374021, 0.36843663],
        ...,
        [0.29638808, 0.06571188, 0.06658522, 0.39548448, 0.45114586],
        [0.81430213, 0.084246  , 0.06719609, 0.39548448, 0.45114586],
        [0.57246623, 0.10362258, 0.06108735, 0.39548448, 0.45114586]],

       [[0.3301167 , 0.57792755, 0.62797801, 0.19374021, 0.36843663],
        [0.86874731, 0.45492839, 0.76786805, 0.19374021, 0.36843663],
        [0.61723798, 0.47177759, 0.77886378, 0.19374021, 0.36843663],
        ...,
        [0.81430213, 0.084246  , 0.06719609, 0.39548448, 0.45114586],
        [0.57246623, 0.10362258, 0.06108735, 0.39548448, 0.45114586],
        [0.9278366 , 0.12299916, 0.03665241, 0.39548448, 0.45114586]],

       [[0.86874731, 0.45492839, 0.76786805, 0.19374021, 0.36843663],
        [0.61723798, 0.47177759, 0.77886378, 0.19374021, 0.3

In [186]:
forecast

array([[-0.05787401],
       [-0.05316967],
       [-0.04670049],
       [-0.04034681],
       [-0.03502016],
       [-0.03084862],
       [-0.02761944],
       [-0.02511154],
       [-0.02462105],
       [-0.0265068 ],
       [-0.03000837],
       [-0.03440547],
       [-0.03896206],
       [-0.04267142]], dtype=float32)

In [191]:
all_history

{'AE': <keras.callbacks.History at 0x7f946b7e1b70>,
 'CSA4000': <keras.callbacks.History at 0x7f946a3aef80>,
 'CSA4000(PCA)': <keras.callbacks.History at 0x7f946a0c7490>,
 'CSA5000': <keras.callbacks.History at 0x7f9465a5d450>,
 'PEMA-500FR': <keras.callbacks.History at 0x7f94612ca770>,
 'PEMA-580FX': <keras.callbacks.History at 0x7f95720f2380>,
 'PEMA-CR1000': <keras.callbacks.History at 0x7f9529eb6560>,
 'PEMA-CSA5000': <keras.callbacks.History at 0x7f9518f85b40>,
 'PEMA-HR1000': <keras.callbacks.History at 0x7f9518c1ee60>,
 'PEMA-HR1000S': <keras.callbacks.History at 0x7f9518b30f70>,
 'PEMA-HR1500': <keras.callbacks.History at 0x7f94f85eed70>,
 'PEMA-PCM2000': <keras.callbacks.History at 0x7f94dc863ca0>,
 'PEMA-PCM2000B': <keras.callbacks.History at 0x7f94f8371a80>,
 'PEMA-PCR3000E': <keras.callbacks.History at 0x7f94b06b8fa0>,
 'PEMA-PCR3000N': <keras.callbacks.History at 0x7f94aed421d0>,
 'PEMA-PR1000': <keras.callbacks.History at 0x7f94a9ebb550>,
 'PEMA-PR2000': <keras.callbacks.

In [188]:
n_future = 14
forcast_period_dates = pd.date_range('2021-04-19', periods = n_future, freq = '7d').tolist()

forecast = all_history['PEMA-HR1000'].predict(pp[-n_future:])
forecast_copies = np.repeat(forecast, product_df.get('PEMA-HR1000').shape[1],axis=-1) # np.repeat(a, n) a를 n개 복사, axis=-1 현재 배열의 마지막 axis
y_pred_future = scaler.inverse_transform(forecast_copies)[:,0]

forecast_dataes = []

for time_i in forcast_period_dates:
    forecast_dataes.append(time_i.date())
    len_forecast = len(forecast[:,-1])#n_future 만큼의 길이
    forecast_dates = pd.date_range(forcast_period_dates[1], periods=len_forecast, freq = str('7d')).tolist()

    df_forecast = pd.DataFrame({'week': np.array(pd.date_range(forcast_period_dates[1],  periods=len_forecast, freq = str('7d')).tolist()), 'sold_quant':y_pred_future})
    df_forecast['week'] = pd.to_datetime(df_forecast['week'])

AttributeError: ignored

In [107]:
all_testY = {}
all_history = {}
all_predcition = {}
all_mse = {}
all_r2 = {}

for product_name ,predicton_df in
BATCH_SIZE = 32
time_steps = n_past
unit = 128

my_GRU_model = Sequential() # 순차적으로 레이블 생성

my_GRU_model.add(GRU(units=unit, return_sequences=True, activation='tanh')) # 유닛수설정,출력시퀀스반환, 탄젠트 함수이용

my_GRU_model.add(Dropout(0.1)) # 드롭아웃 과적합 줄이기

my_GRU_model.add(GRU(units=unit, return_sequences=True, activation='tanh'))

my_GRU_model.add(Dropout(0.1))

my_GRU_model.add(GRU(units=unit, return_sequences=True, activation='tanh'))

my_GRU_model.add(Dropout(0.1))


my_GRU_model.add(GRU(units=unit, activation='tanh'))


my_GRU_model.add(Dense(units=1))


my_optimizer = Adam(learning_rate=0.002)


my_GRU_model.compile(optimizer=my_optimizer, loss='huber_loss', metrics=['mse'])



history = my_GRU_model.fit(trainX, trainY, epochs=100, validation_split=0.2, batch_size=BATCH_SIZE, verbose=0)

predictions = my_GRU_model.predict(trainX)  # 모델을 사용하여 예측

mse = mean_squared_error(trainY, predictions)
print("Mean Squared Error:", mse)

r2 = r2_score(trainY, predictions)
print("R2 Score:", r2)

4/4 [==============================] - 1s 4ms/step
Mean Squared Error: 0.002671588476467101
R2 Score: 0.9671262305727916


In [83]:
from sklearn.metrics import mean_squared_error, r2_score
mse = mean_squared_error(trainY, predictions)
print("Mean Squared Error:", mse)

r2 = r2_score(trainY, predictions)
print("R2 Score:", r2)

Mean Squared Error: 0.0017248706962432023
R2 Score: 0.9787755479335533


In [92]:
trainX.shape
trainY.shape

(108, 1)

In [18]:
import pandas as pd
import numpy as np
from keras.models import Sequential
from keras.layers import Dense, SimpleRNN, GRU
from tensorflow.keras.losses import Huber
# from keras.optimizers import SGD
from keras.layers import Dropout
from keras.optimizers import Adam
from sklearn.preprocessing import MinMaxScaler





BATCH_SIZE = 32
time_steps = n_past
unit = 128

my_GRU_model = Sequential() # 순차적으로 레이블 생성

my_GRU_model.add(GRU(units=unit, return_sequences=True, activation='tanh')) # 유닛수설정,출력시퀀스반환, 탄젠트 함수이용

my_GRU_model.add(Dropout(0.1)) # 드롭아웃 과적합 줄이기

my_GRU_model.add(GRU(units=unit, return_sequences=True, activation='tanh'))

my_GRU_model.add(Dropout(0.1))

my_GRU_model.add(GRU(units=unit, return_sequences=True, activation='tanh'))

my_GRU_model.add(Dropout(0.1))


my_GRU_model.add(GRU(units=unit, activation='tanh'))


my_GRU_model.add(Dense(units=1))


my_optimizer = Adam(learning_rate=0.002)


my_GRU_model.compile(optimizer=my_optimizer, loss='huber_loss', metrics=['mse'])


history = my_GRU_model.fit(trainX, trainY, epochs=200, validation_split=0.2, batch_size=BATCH_SIZE, verbose=0)

predictions = my_GRU_model.predict(trainX)  # 모델을 사용하여 예측

mse = mean_squared_error(trainY, predictions)
print("Mean Squared Error:", mse)

r2 = r2_score(trainY, predictions)
print("R2 Score:", r2)

n_future = 14
forcast_period_dates = pd.date_range('2021-04-19', periods = n_future, freq = '7d').tolist()

forecast = my_GRU_model.predict(trainX[-n_future:])
forecast_copies = np.repeat(forecast, data_scaled.shape[1],axis=-1) # np.repeat(a, n) a를 n개 복사, axis=-1 현재 배열의 마지막 axis
y_pred_future = scaler.inverse_transform(forecast_copies)[:,0]

len_forecast = len(forecast[:,-1])#n_future 만큼의 길이
forecast_dates = pd.date_range(forcast_period_dates[1], periods=len_forecast, freq = str('7d')).tolist()

forecast_dataes = []
for time_i in forcast_period_dates:
  forecast_dates.append(time_i.date())


df_forecast = pd.DataFrame({'week': np.array(pd.date_range(forcast_period_dates[1],  periods=len_forecast, freq = str('7d')).tolist()), 'sold_quant':y_pred_future})
df_forecast['week'] = pd.to_datetime(df_forecast['week'])

df_forecast

ValueError: ignored

In [6]:
df = df[df['prodname'] == 'PEMA-HR1000']
df2 = df[['week', 'tem_avg', 'hum_avg', 'sold_quant', '국내건설수주액', '국내기성액']].copy()


In [7]:
data = df2.copy()
data = df2.set_index('week',inplace = True)
data = df2.sort_values('week')

In [ ]:
# 'sold_quant' 컬럼을 추출
sold_quant_column = data['sold_quant']

# 'sold_quant' 컬럼을 제외한 나머지 컬럼들
other_columns = data.drop(columns=['sold_quant'])

# 'sold_quant' 컬럼을 맨 앞으로 배치한 데이터프레임 생성
data= pd.concat([sold_quant_column, other_columns], axis=1)

data